## This notebook defines a function "create_lane_closure" to close lanes in SUMO network
The function creates an xml file that is used as additional-file with a SUMO simulation

In [11]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
import random
from xml.dom import minidom

### Example Reroute Additional File

In [12]:
'''
<additional>
  <rerouter id="try_1" edges="1527 -1527" probability="0.1">
      <interval begin="1" end="1000">
         <closingReroute id="1527" disallow="passenger truck"/>
         <closingReroute id="-1527" disallow="truck"/>
      </interval>
   </rerouter>
</additional>
'''

'\n<additional>\n  <rerouter id="try_1" edges="1527 -1527" probability="0.1">\n      <interval begin="1" end="1000">\n         <closingReroute id="1527" disallow="passenger truck"/>\n         <closingReroute id="-1527" disallow="truck"/>\n      </interval>\n   </rerouter>\n</additional>\n'

In [16]:
'''
reroute_edges: list of edge ids (string) where vehicle rerouting will happen
lanes_to_close: list of lane ids (string) to close
reroute_prob: probability (float) that a vehicle passing a rerouting edge can reroute
disallow_vehs: list of vehicle types (string) to disallow on closed lanes
begin: time (in seconds) when the lane closures start 
end: time (in seconds) when the lane closures end
file_name: name (string) of lane closure additional file
'''

def create_lane_closures(reroute_edges, lanes_to_close, begin, end, reroute_prob = 1, 
                        disallow_vehs = ['passenger'], file_name = 'lane_closures.xml'):
    if len(reroute_edges) == 0 or len(lanes_to_close) == 0:
        print('Missing edge ids where to reroute or lane ids for lane closures')
        return
    elif begin < 0: 
        print('begin second is less than zero')
        return
    else:
        xadditional = Element('additional')

        xrerouter = Element('rerouter')
        xadditional.append(xrerouter)
        xrerouter.set('id','rerouter'+str(len(lanes_to_close)))
        
        i = 1
        edges_str = ""
        for edge in reroute_edges:
            if i == 1: edges_str = edge
            else: edges_str = edges_str + " " + edge
            i = i + 1

        xrerouter.set('edges', edges_str)
        xrerouter.set('probability', str(reroute_prob))

        xinterval = Element('interval')
        xrerouter.append(xinterval)
        xinterval.set('begin', str(begin))
        xinterval.set('end', str(end))

        for lane in lanes_to_close:
            xclosing = Element('closingLaneReroute')
            xinterval.append(xclosing)
            xclosing.set('id', lane)
            
            j = 1;
            for veh in disallow_vehs:
                if j == 1: veh_str = veh
                else: veh_str = veh_str + " " + veh
                j = j + 1
            xclosing.set('disallow', veh_str)

        # Writing down the xml file
        configfile = file_name
        with open(configfile, 'w') as f:
            f.write(minidom.parseString(ET.tostring(xadditional)).toprettyxml(indent="\t"))

In [21]:
#Example to use the function
reroute_edges = ['1527', '-1527']
lanes_to_close = ['1527_0', '-1527_0']
# disallow_vehs = ['passenger']
# reroute_prob = 0.5
begin = 0
end = 1000
name = 'add.xml'

In [22]:
#calling create_lane_closure function
create_lane_closures(reroute_edges, lanes_to_close, begin, end)